# 00A — Index Data Load & Validation

**Purpose**: Load all Indian index CSVs, validate schema/dates/continuity, convert daily → monthly

**Inputs**: `../data_raw/index/*_day.csv`

**Outputs**:
- `../data_processed/india_indices_daily.parquet`
- `../data_processed/india_indices_monthly.parquet`

---

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Paths
RAW_INDEX_PATH = Path('../data_raw/index')
PROCESSED_PATH = Path('../data_processed')
PROCESSED_PATH.mkdir(exist_ok=True)

print(f"Index files found: {len(list(RAW_INDEX_PATH.glob('*_day.csv')))}")

Index files found: 21


## 1. Load All Index CSVs

In [2]:
def load_index_csv(filepath: Path) -> pd.DataFrame:
    """Load a single index CSV with proper date parsing and duplicate handling."""
    df = pd.read_csv(filepath)
    
    # Identify date column (usually 'Date' or first column)
    date_col = 'Date' if 'Date' in df.columns else df.columns[0]
    
    # Parse dates
    df[date_col] = pd.to_datetime(df[date_col], dayfirst=True, errors='coerce')
    df = df.dropna(subset=[date_col])
    df = df.set_index(date_col)
    df.index.name = 'Date'
    
    # === HANDLE DUPLICATES ===
    if df.index.duplicated().any():
        df = df[~df.index.duplicated(keep='last')]
        
    # Extract index name from filename
    index_name = filepath.stem.replace('_day', '').replace(' ', '_')
    
    # Keep only OHLC + Volume if available
    ohlc_cols = ['Open', 'High', 'Low', 'Close']
    available_cols = [c for c in ohlc_cols if c in df.columns]
    
    if 'Close' not in df.columns:
        # Try alternative column names
        for alt in ['close', 'CLOSE', 'Closing Price']:
            if alt in df.columns:
                df['Close'] = df[alt]
                available_cols.append('Close')
                break
    
    df = df[available_cols].copy()
    
    # Rename columns with index prefix for later merging
    df.columns = [f"{index_name}_{c}" for c in df.columns]
    
    return df

# Load all indices
all_indices = {}
for csv_file in sorted(RAW_INDEX_PATH.glob('*_day.csv')):
    index_name = csv_file.stem.replace('_day', '').replace(' ', '_')
    try:
        df = load_index_csv(csv_file)
        all_indices[index_name] = df
        print(f"✓ {index_name}: {len(df)} rows, {df.index.min().date()} to {df.index.max().date()}")
    except Exception as e:
        print(f"✗ {index_name}: {e}")

print(f"\nTotal indices loaded: {len(all_indices)}")

✓ NIFTY_100: 1434 rows, 2015-01-01 to 2026-12-01
✓ NIFTY_200: 1434 rows, 2015-01-01 to 2026-12-01


✓ NIFTY_500: 1087 rows, 2015-01-01 to 2026-12-01
✓ NIFTY_50: 1088 rows, 2015-01-01 to 2026-12-01
✓ NIFTY_ALPHA_50: 1088 rows, 2015-01-01 to 2026-12-01
✓ NIFTY_AUTO: 1426 rows, 2015-01-01 to 2026-12-01


✓ NIFTY_BANK: 1091 rows, 2015-01-01 to 2026-12-01
✓ NIFTY_COMMODITIES: 1369 rows, 2015-01-01 to 2026-12-01
✓ NIFTY_CONSR_DURBL: 1088 rows, 2015-01-01 to 2026-12-01


✓ NIFTY_CONSUMPTION: 1088 rows, 2015-01-01 to 2026-12-01
✓ NIFTY_CPSE: 1127 rows, 2015-01-01 to 2026-12-01
✓ NIFTY_ENERGY: 1090 rows, 2015-01-01 to 2026-12-01
✓ NIFTY_FIN_SERVICE: 1090 rows, 2015-01-01 to 2026-12-01
✓ NIFTY_FMCG: 1090 rows, 2015-01-01 to 2026-12-01
✓ NIFTY_GS_COMPSITE: 797 rows, 2015-09-11 to 2026-12-01
✓ NIFTY_HEALTHCARE: 1083 rows, 2015-01-01 to 2026-12-01


✓ NIFTY_IND_DIGITAL: 1083 rows, 2015-01-01 to 2026-12-01
✓ NIFTY_INDIA_MFG: 1083 rows, 2015-01-01 to 2026-12-01
✓ NIFTY_INFRA: 1065 rows, 2015-01-01 to 2026-12-01
✓ NIFTY_IT: 1091 rows, 2015-01-01 to 2026-12-01


✓ NIFTY_LARGEMID250: 1083 rows, 2015-01-01 to 2026-12-01

Total indices loaded: 21


## 2. Schema Validation

In [3]:
def validate_schema(df: pd.DataFrame, index_name: str) -> dict:
    """Validate schema and return validation report."""
    close_col = f"{index_name}_Close"
    
    report = {
        'index': index_name,
        'rows': len(df),
        'has_close': close_col in df.columns,
        'null_close_pct': df[close_col].isna().mean() * 100 if close_col in df.columns else 100,
        'start_date': df.index.min(),
        'end_date': df.index.max(),
        'years': (df.index.max() - df.index.min()).days / 365.25 if not df.empty else 0
    }
    return report

if all_indices:
    validation_reports = [validate_schema(df, name) for name, df in all_indices.items()]
    validation_df = pd.DataFrame(validation_reports)
    print("Validation Summary:")
    print(validation_df.sort_values('years', ascending=False) if not validation_df.empty else "No indices validated")
else:
    print("No indices loaded for validation")

Validation Summary:
                index  rows  has_close  null_close_pct          start_date  \
0           NIFTY_100  1434       True             0.0 2015-01-01 00:00:00   
1           NIFTY_200  1434       True             0.0 2015-01-01 00:00:00   
2           NIFTY_500  1087       True             0.0 2015-01-01 00:00:00   
3            NIFTY_50  1088       True             0.0 2015-01-01 00:00:00   
4      NIFTY_ALPHA_50  1088       True             0.0 2015-01-01 00:00:00   
5          NIFTY_AUTO  1426       True             0.0 2015-01-01 00:00:00   
7   NIFTY_COMMODITIES  1369       True             0.0 2015-01-01 00:00:00   
8   NIFTY_CONSR_DURBL  1088       True             0.0 2015-01-01 00:00:00   
9   NIFTY_CONSUMPTION  1088       True             0.0 2015-01-01 00:00:00   
11       NIFTY_ENERGY  1090       True             0.0 2015-01-01 00:00:00   
10         NIFTY_CPSE  1127       True             0.0 2015-01-01 00:00:00   
12  NIFTY_FIN_SERVICE  1090       True      

## 3. Date Continuity Check

In [4]:
def check_date_gaps(df: pd.DataFrame, max_gap_days: int = 5) -> list:
    """Find gaps larger than max_gap_days (excluding weekends/holidays)."""
    if df.empty:
        return []
    # Sort index just in case
    df = df.sort_index()
    date_diff = df.index.to_series().diff()
    large_gaps = date_diff[date_diff > pd.Timedelta(days=max_gap_days)]
    return [(idx, gap.days) for idx, gap in large_gaps.items()]

# Check gaps for each index
gap_report = {}
for name, df in all_indices.items():
    gaps = check_date_gaps(df)
    if gaps:
        gap_report[name] = len(gaps)
        
if gap_report:
    print("Indices with date gaps > 5 days:")
    for name, count in gap_report.items():
        print(f"  {name}: {count} gaps")
else:
    print("✓ No significant date gaps found in any index")

Indices with date gaps > 5 days:
  NIFTY_100: 139 gaps
  NIFTY_200: 139 gaps
  NIFTY_500: 139 gaps
  NIFTY_50: 139 gaps
  NIFTY_ALPHA_50: 139 gaps
  NIFTY_AUTO: 139 gaps
  NIFTY_BANK: 139 gaps
  NIFTY_COMMODITIES: 139 gaps
  NIFTY_CONSR_DURBL: 139 gaps
  NIFTY_CONSUMPTION: 139 gaps
  NIFTY_CPSE: 139 gaps
  NIFTY_ENERGY: 139 gaps
  NIFTY_FIN_SERVICE: 139 gaps
  NIFTY_FMCG: 139 gaps
  NIFTY_GS_COMPSITE: 104 gaps
  NIFTY_HEALTHCARE: 139 gaps
  NIFTY_IND_DIGITAL: 139 gaps
  NIFTY_INDIA_MFG: 139 gaps
  NIFTY_INFRA: 139 gaps
  NIFTY_IT: 139 gaps
  NIFTY_LARGEMID250: 139 gaps


## 4. Merge All Indices (Daily)

In [5]:
if all_indices:
    # Merge all indices on date (outer join to preserve all dates)
    daily_merged = pd.concat(all_indices.values(), axis=1, join='outer')
    daily_merged = daily_merged.sort_index()

    # Keep only Close prices for simplicity (can expand later)
    close_cols = [c for c in daily_merged.columns if c.endswith('_Close')]
    daily_close = daily_merged[close_cols].copy()

    # Rename columns to just index names
    daily_close.columns = [c.replace('_Close', '') for c in daily_close.columns]

    print(f"Daily data shape: {daily_close.shape}")
    print(f"Date range: {daily_close.index.min().date()} to {daily_close.index.max().date()}")
    print(daily_close.tail())
else:
    print("No indices loaded to merge")

Daily data shape: (1436, 21)
Date range: 2015-01-01 to 2026-12-01
            NIFTY_100  NIFTY_200  NIFTY_500  NIFTY_50  NIFTY_ALPHA_50  \
Date                                                                    
2026-06-01   26808.75   14615.75   24010.50  26178.70        50958.70   
2026-07-01   26781.80   14615.85   24016.60  26140.75        51064.45   
2026-08-01   26462.60   14421.05   23680.10  25876.85        50076.35   
2026-09-01   26252.95   14306.95   23467.35  25683.30        49332.75   
2026-12-01   26357.40   14352.10   23510.45  25790.25        49217.60   

            NIFTY_AUTO  NIFTY_BANK  NIFTY_COMMODITIES  NIFTY_CONSR_DURBL  \
Date                                                                       
2026-06-01    28922.35    60118.40            9700.20           37699.00   
2026-07-01    28692.40    59990.85            9669.70           38337.95   
2026-08-01    28414.95    59686.50            9437.25           38037.60   
2026-09-01    28087.65    59251.55        

## 5. Convert Daily → Monthly

In [6]:
def daily_to_monthly(df: pd.DataFrame) -> pd.DataFrame:
    """Convert daily prices to monthly (last business day of month)."""
    if df.empty:
        return df
    # Resample to month-end, taking last available value
    monthly = df.resample('ME').last()
    
    # Drop rows where all values are NaN
    monthly = monthly.dropna(how='all')
    
    return monthly

if 'daily_close' in locals() and not daily_close.empty:
    monthly_close = daily_to_monthly(daily_close)

    print(f"Monthly data shape: {monthly_close.shape}")
    print(f"Date range: {monthly_close.index.min().date()} to {monthly_close.index.max().date()}")
    print(monthly_close.tail())
else:
    print("No daily close data for monthly conversion")

Monthly data shape: (140, 21)
Date range: 2015-01-31 to 2026-12-31
            NIFTY_100  NIFTY_200  NIFTY_500  NIFTY_50  NIFTY_ALPHA_50  \
Date                                                                    
2026-06-30   26808.75   14615.75   24010.50  26178.70        50958.70   
2026-07-31   26781.80   14615.85   24016.60  26140.75        51064.45   
2026-08-31   26462.60   14421.05   23680.10  25876.85        50076.35   
2026-09-30   26252.95   14306.95   23467.35  25683.30        49332.75   
2026-12-31   26357.40   14352.10   23510.45  25790.25        49217.60   

            NIFTY_AUTO  NIFTY_BANK  NIFTY_COMMODITIES  NIFTY_CONSR_DURBL  \
Date                                                                       
2026-06-30    28922.35    60118.40            9700.20           37699.00   
2026-07-31    28692.40    59990.85            9669.70           38337.95   
2026-08-31    28414.95    59686.50            9437.25           38037.60   
2026-09-30    28087.65    59251.55       

## 6. Calculate Monthly Returns

In [7]:
if 'monthly_close' in locals() and not monthly_close.empty:
    # Calculate monthly returns (percentage change)
    monthly_returns = monthly_close.pct_change() * 100
    monthly_returns = monthly_returns.dropna(how='all')

    print(f"Monthly returns shape: {monthly_returns.shape}")
    print("\nSample returns (last 5 months):")
    print(monthly_returns.tail())
else:
    print("No monthly close data for returns calculation")

Monthly returns shape: (139, 21)

Sample returns (last 5 months):


            NIFTY_100  NIFTY_200  NIFTY_500  NIFTY_50  NIFTY_ALPHA_50  \
Date                                                                    
2026-06-30  -0.183929  -0.185414  -0.199306 -0.272759        0.271839   
2026-07-31  -0.100527   0.000684   0.025406 -0.144965        0.207521   
2026-08-31  -1.191854  -1.332800  -1.401114 -1.009535       -1.935006   
2026-09-30  -0.792250  -0.791205  -0.898434 -0.747966       -1.484933   
2026-12-31   0.397860   0.315581   0.183659  0.416418       -0.233415   

            NIFTY_AUTO  NIFTY_BANK  NIFTY_COMMODITIES  NIFTY_CONSR_DURBL  \
Date                                                                       
2026-06-30    0.097598    0.123576          -0.375894           0.117515   
2026-07-31   -0.795060   -0.212165          -0.314427           1.694873   
2026-08-31   -0.966981   -0.507327          -2.403901          -0.783427   
2026-09-30   -1.151858   -0.728724          -0.743331          -1.142685   
2026-12-31   -0.270226    0.3357

## 7. Data Quality Summary

In [8]:
if 'monthly_close' in locals() and not monthly_close.empty:
    # Coverage summary
    coverage = pd.DataFrame({
        'Start': monthly_close.apply(lambda x: x.first_valid_index()),
        'End': monthly_close.apply(lambda x: x.last_valid_index()),
        'Months': monthly_close.notna().sum(),
        'Missing_Pct': (monthly_close.isna().sum() / len(monthly_close) * 100).round(1)
    })
    coverage = coverage.sort_values('Months', ascending=False)

    print("Index Coverage Summary:")
    print(coverage)
else:
    print("No monthly close data for coverage summary")

Index Coverage Summary:
                       Start        End  Months  Missing_Pct
NIFTY_100         2015-01-31 2026-12-31     140          0.0
NIFTY_200         2015-01-31 2026-12-31     140          0.0
NIFTY_500         2015-01-31 2026-12-31     140          0.0
NIFTY_50          2015-01-31 2026-12-31     140          0.0
NIFTY_ALPHA_50    2015-01-31 2026-12-31     140          0.0
NIFTY_AUTO        2015-01-31 2026-12-31     140          0.0
NIFTY_BANK        2015-01-31 2026-12-31     140          0.0
NIFTY_COMMODITIES 2015-01-31 2026-12-31     140          0.0
NIFTY_CONSR_DURBL 2015-01-31 2026-12-31     140          0.0
NIFTY_CONSUMPTION 2015-01-31 2026-12-31     140          0.0
NIFTY_CPSE        2015-01-31 2026-12-31     140          0.0
NIFTY_ENERGY      2015-01-31 2026-12-31     140          0.0
NIFTY_FIN_SERVICE 2015-01-31 2026-12-31     140          0.0
NIFTY_FMCG        2015-01-31 2026-12-31     140          0.0
NIFTY_HEALTHCARE  2015-01-31 2026-12-31     140          0.0


## 8. Export to Parquet

In [9]:
if 'daily_close' in locals() and not daily_close.empty:
    # Save daily close prices
    daily_output = PROCESSED_PATH / 'india_indices_daily.parquet'
    daily_close.to_parquet(daily_output)
    print(f"✓ Saved daily data: {daily_output}")

if 'monthly_close' in locals() and not monthly_close.empty:
    # Save monthly close prices
    monthly_output = PROCESSED_PATH / 'india_indices_monthly.parquet'
    monthly_close.to_parquet(monthly_output)
    print(f"✓ Saved monthly data: {monthly_output}")

if 'monthly_returns' in locals() and not monthly_returns.empty:
    # Save monthly returns
    returns_output = PROCESSED_PATH / 'india_indices_monthly_returns.parquet'
    monthly_returns.to_parquet(returns_output)
    print(f"✓ Saved monthly returns: {returns_output}")

✓ Saved daily data: ..\data_processed\india_indices_daily.parquet


✓ Saved monthly data: ..\data_processed\india_indices_monthly.parquet


✓ Saved monthly returns: ..\data_processed\india_indices_monthly_returns.parquet

## 9. Validation Complete ✓

**Outputs produced:**
- `india_indices_daily.parquet` — Daily close prices for all indices
- `india_indices_monthly.parquet` — Monthly close prices (month-end)
- `india_indices_monthly_returns.parquet` — Monthly percentage returns

**Next notebook:** `00B_rbi_macro_data_load_validation.ipynb`